In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta
from missingpy import MissForest #impute missing value
from sklearn.preprocessing import MinMaxScaler #standardized data
import random
import matplotlib.pyplot as plt

#deep learning package
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.models import Model

import keras
from keras import layers

import preprocessing
from model import CNN_LSTM

In [3]:
impute_data_frame2 = pd.read_csv('impute_data_frame2.csv')
select_survey = pd.read_csv('select_survey.csv')

Data augumentation: for Fitbit only

In [4]:
def add_gaussian_noise(time_series, mean=0.0, stddev=1.0):
    """
    Adds Gaussian noise to a time series.
     Options:
     time_series (array-like): A time series to which noise is added.
     mean (float): The average value of the noise. Default is 0.0.
     stddev (float): Standard deviation of noise. Default is 1.0.

     Returns:
     noisy_series (np.array): Time series with added noise.
     """
     # Gaussian noise generation
    noise = np.random.normal(mean, stddev, len(time_series))

    # Adding noise to the original time series
    noisy_series = time_series + noise

    return noisy_series

In [5]:
data_augumentation = impute_data_frame2.copy()
non_need_augumentation = ['survey_date', 'coach_date', 'date']
data_augumentation.drop(columns = non_need_augumentation, inplace = True)

total_ids = data_augumentation['Id'].unique()
after_augumentation = pd.DataFrame()

for id in total_ids:
    individual_data = data_augumentation[data_augumentation['Id'] == id]
    individual_data = individual_data.drop(columns = 'Id').copy()

    augmented_data = individual_data.apply(lambda col: add_gaussian_noise(col, mean=0.0, stddev=0.05))
    augmented_data['Id'] = id
    after_augumentation = pd.concat([after_augumentation, augmented_data])

after_augumentation[['survey_date', 'coach_date', 'date']] = impute_data_frame2[['survey_date', 'coach_date', 'date']]
after_augumentation.set_index('Id', inplace = True)
after_augumentation.reset_index(inplace = True)

Merge fitbit data with survey data

In [ ]:
#category mental health, three levels
# The average of the five responses were used for data analysis
select_survey['mental_health_1'] = select_survey[['nervous_v1', 'down_v1', 'calm_v1','blue_v1', 'happy_v1']].mean(axis = 1)
select_survey['mental_health_2'] = select_survey[['nervous_v2', 'down_v2', 'calm_v2','blue_v2', 'happy_v2']].mean(axis = 1)

#determine the median of mental health > I want to make it to be multicategory: low, controllable, high
mental_health1_median = select_survey['mental_health_1'].median()
mental_health2_median = select_survey['mental_health_2'].median()

#transfer the variable to category
select_survey['mental_health_1'] = select_survey['mental_health_1'].apply(lambda value: 0 if value < mental_health1_median else 1)
select_survey['mental_health_2'] = select_survey['mental_health_2'].apply(lambda value: 0 if value < mental_health1_median else 1)

In [7]:
#recode demongraphic variable
all_fitbit_id = data_augumentation['Id'].unique()

var = ['demographics_race___1','demographics_race___2','demographics_race___3','demographics_race___4',
       'demographics_race___5','demographics_race___6','demographics_race___7']
recode_demographic_variable = select_survey[var]
recode_demographic_variable['demographic'] = np.nan
recode_demographic_variable['Id'] = select_survey['Id']

#1: demographics_race___1 = 1
#2: demographics_race___2 = 1
#3: demographics_race___3 = 3
#4: demographics_race___4 = 4
#5: demographics_race___5 = 5
#6: demographics_race___6 = 6
#7: demographics_race___7 = 7
#8: multiple_race = 8

# Recoding logic
for i in all_fitbit_id:
    row = recode_demographic_variable[recode_demographic_variable['Id'] == i][var]
    # Check if there are multiple races selected
    if row.sum(axis=1).iloc[0] > 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 8  # Multiple races
    elif row['demographics_race___1'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 1
    elif row['demographics_race___2'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 2
    elif row['demographics_race___3'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 3
    elif row['demographics_race___4'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 4
    elif row['demographics_race___5'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 5
    elif row['demographics_race___6'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 6
    elif row['demographics_race___7'].iloc[0] == 1:
        recode_demographic_variable.loc[recode_demographic_variable['Id'] == i, 'demographic'] = 7

select_survey['demographics_race'] =  recode_demographic_variable['demographic']

used_variables = ['Id', 'current_status','demographics_age','demographics_sex','demographics_sexorient',
                  'demographics_ethnicity', 'demographics_immigration','demographics_race',
                  'demographics_education', 'demographics_sorority']

select_data = select_survey[used_variables]

#merge Fitbit data with survey data
fitbit_survey = pd.merge(after_augumentation, select_data, on = 'Id', how = 'left')

#move the date time
fitbit_survey.set_index('date', inplace=True)
fitbit_survey.reset_index(inplace=True)

C:\Users\karri\AppData\Local\Temp\ipykernel_27244\2603091222.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['demographic'] = np.nan
C:\Users\karri\AppData\Local\Temp\ipykernel_27244\2603091222.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recode_demographic_variable['Id'] = select_survey['Id']


In [8]:
# impute time sequence data
miss_imputation3 = preprocessing.impute_missing(fitbit_survey)
#keep only one date
impute_data_frame3 = miss_imputation3.copy()
impute_data_frame3 = impute_data_frame3.reset_index().drop(columns = 'index')
date = impute_data_frame3.iloc[:, 0]
impute_data_frame3 = impute_data_frame3.drop(columns=['date'])
impute_data_frame3['date'] = date

d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning original dataset.
  warnings.warn("No missing value located; returning original "
d:\EID-Download\02 Academic\Python38\lib\site-packages\missingpy\missforest.py:528: UserWarning: No missing value located; returning origina

In [9]:
# academic day: weekday except break
# non-academic day: weekend, fallbreak, winterbreak, spring semester, spring break, summer holiday)

#weekend:
#----10.07 saturaday - non-academic day
#----10.08 sunday - non-academic day

#fall break "2023-10-8 :2023-10-15"
#winter break '2023-11-15:'2023-01-08'
#martine luda:  2024 January 15
#president date:  2024 February 19
#spring break: 2024 March 3-10
#summer break: '2023-5-1: 2023-8-21'

fall_break = pd.date_range(start='2023-10-8', end='2023-10-15')
winter_break = pd.date_range(start='2023-11-15', end='2024-01-08')
other_days = ['2024-01-15', '2024-02-19']
other_days = pd.DatetimeIndex(pd.to_datetime(other_days))
spring_break = pd.date_range(start='2024-03-03', end='2024-03-10')
summer_break = pd.date_range(start='2023-5-1', end='2023-8-21')


earlist_date = impute_data_frame3['date'].min()
latest_date = impute_data_frame3['date'].max()

# Generate a date range between the start and end dates
determine_weekend = pd.date_range(start=earlist_date, end=latest_date)

date_info = pd.DataFrame({
    'date': determine_weekend,
    'is_weekend': determine_weekend.weekday.isin([5, 6])  # True for Saturday (5) and Sunday (6)
})

# Filter to display only weekend days
weekends = date_info[date_info['is_weekend']]['date'].values
weekends = pd.DatetimeIndex(pd.to_datetime(weekends))

#combine all the non-academic day
non_academic_days = fall_break.union(winter_break).union(other_days).union(spring_break).union(summer_break).union(weekends)

#recode the date data
impute_data_frame3['date_code'] = impute_data_frame3['date'].apply(lambda x:0 if x in non_academic_days else 1)

fitbit_survey2 = impute_data_frame3.drop(columns = 'date')

Padding

In [10]:
mental_health1 =  select_survey['mental_health_1'].values
mental_health2 =  select_survey['mental_health_2'].values

ids = fitbit_survey2['Id'].unique()
max_short_day = 0
max_long_day = 0
for id in ids:
    individual_data_short = fitbit_survey2[(fitbit_survey2['Id'] == id) & (fitbit_survey2['survey_date'] < 2)]
    individual_data_long = fitbit_survey2[(fitbit_survey2['Id'] == id) & (fitbit_survey2['survey_date'] < 4)]
    if len(individual_data_short) > max_short_day:
        max_short_day = len(individual_data_short)

    if len(individual_data_long) > max_long_day:
        max_long_day = len(individual_data_long)

# List to store padded matrices for each ID
padded_matrices = []

for id_val in ids:
    # Filter data for the current `id_val` and specified condition
    id_data = fitbit_survey2[(fitbit_survey2['Id'] == id_val) & (fitbit_survey2['survey_date'] < 2)]

    max_rows = max_short_day
    col = id_data.shape[1]
    
    # Convert to matrix (numpy array) without 'Id' and 'survey_date' columns
    id_matrix = id_data.drop(columns=['Id', 'survey_date']).values
    
    # Pad each matrix to the target shape (max_rows, max_cols)
    padded_id_matrix = np.pad(id_matrix, ((0, max_rows - id_matrix.shape[0]), (0, col - id_matrix.shape[1])), mode='constant', constant_values=0) 
    #pad the dataset with the same length with 0 and the padding value will be ignored by masking
    
    # Append the padded matrix to the list
    padded_matrices.append(padded_id_matrix) #add 2D matrix to 3D

# Stack all matrices into a single 3D array (number of IDs, max_rows, max_cols)
final_padded_matrix = np.stack(padded_matrices, axis=0) 


# List to store padded matrices for each ID
padded_matrices2 = []

for id_val in ids:
    # Filter data for the current `id_val` and specified condition
    id_data = fitbit_survey2[(fitbit_survey2['Id'] == id_val) & (fitbit_survey2['survey_date'] < 4)]

    max_rows = max_long_day
    col = id_data.shape[1]
    
    # Convert to matrix (numpy array) without 'Id' and 'survey_date' columns
    id_matrix = id_data.drop(columns=['Id', 'survey_date']).values
    
    # Pad each matrix to the target shape (max_rows, max_cols)
    padded_id_matrix = np.pad(id_matrix, ((0, max_rows - id_matrix.shape[0]), (0, col - id_matrix.shape[1])), mode='constant', constant_values=0) 
    #pad the dataset with the same length with 0 and the padding value will be ignored by masking
    
    # Append the padded matrix to the list
    padded_matrices2.append(padded_id_matrix) #add 2D matrix to 3D

# Stack all matrices into a single 3D array (number of IDs, max_rows, max_cols)
final_padded_matrix2 = np.stack(padded_matrices2, axis=0) 

Prepare matrix for training

In [11]:
X_short_data = final_padded_matrix.copy()
X_long_data = final_padded_matrix2.copy()
y_short_data = mental_health1.copy()
y_long_data = mental_health2.copy()

In [12]:
#standardize the data
scaler = MinMaxScaler()

X_reshpaed = X_short_data.reshape(-1, X_short_data.shape[2])
X_standardized = scaler.fit_transform(X_reshpaed)
X_short_standardized = X_standardized.reshape(X_short_data.shape)

X_reshpaed2 = X_long_data.reshape(-1, X_long_data.shape[2])
X_standardized2 = scaler.fit_transform(X_reshpaed2)
X_short_standardized2 = X_standardized2.reshape(X_long_data.shape) #

In [ ]:
def train_test_split(dataX, datay, shuffle = True, train_percentage = 0.7, val_percentage = 0.15):
    """
    parameters:
    dataX: the feature data
    datay: the labels
    shuffle: whether to shuffle the data before splitting
    train_percentage: proportion of data to use for the training set
    val_percentage: proportion of data to use for the validation set

    returns
    data for training, testing, and validating
    """
    if shuffle:
        random_indices = np.arange(len(dataX))
        np.random.shuffle(random_indices)
        dataX = dataX[random_indices]
        datay = datay[random_indices]
    
    # Compute split indices
    train_end = int(len(dataX) * train_percentage)
    val_end = train_end + int(len(dataX) * val_percentage)
    
    # Split the data
    train_X, train_y = dataX[:train_end], datay[:train_end]
    val_X, val_y = dataX[train_end:val_end], datay[train_end:val_end]
    test_X, test_y = dataX[val_end:], datay[val_end:]
    
    return train_X, train_y, val_X, val_y, test_X, test_y 

In [ ]:
#split data to training, testing, and validating
train_X_data, train_y_data, val_X_data, val_y_data, test_X_data, test_y_data = train_test_split(X_short_standardized, y_short_data)

Tune hyperparameters for CNN-LSTM model

In [ ]:
# Hyperparameter space
hyperparameter_space = {
    'hidden_size': [4, 8, 16, 32, 64],
    'num_layers': [1, 2, 3, 4],
    'learning_rate': [0.01, 0.001, 0.0001],
    'batch_size': [4, 8, 16],
}

# hyperparameter_space = {
#     'embed_size': [16, 32, 64],
#     'conv_input': [8, 16, 32],
#     'input_size': [8, 16, 32],
#     'hidden_size': [32, 64, 128],
#     'num_layers': [1, 2, 3],
#     'batch_size': [16, 32, 64],
#     'learning_rate': [0.01, 0.001, 0.0001]
#     #'num_epochs': [10, 20, 30] #do I need to tune this hyperparameter
# }

#transfer data to torch data
# Convert training data to PyTorch tensors
train_X_new = torch.tensor(train_X_data, dtype=torch.float32)  # Convert train_X to float32 tensor
train_y_new = torch.tensor(train_y_data, dtype=torch.float32)  # Convert train_y to float32 tensor

# Convert testing data to PyTorch tensors
val_X_new = torch.tensor(val_X_data, dtype=torch.float32)    # Convert test_X to float32 tensor
val_y_new = torch.tensor(val_y_data, dtype=torch.float32)    # Convert test_y to float32 tensor

# Generate synthetic data
train_X = train_X_new  # (samples, sequence length, features)
train_y = train_y_new        # Corresponding labels
train_seq_lengths = torch.randint(1, 110, (100,))  # Sequence lengths

val_X = val_X_new
val_y = val_y_new
val_seq_lengths = torch.randint(1, 110, (20,))

# Number of random search trials
num_trials = 10
best_val_loss = float('inf')
best_hyperparams = {}

# Random search loop
for _ in range(num_trials):
    # Randomly sample hyperparameters
    hidden_size = random.choice(hyperparameter_space['hidden_size'])
    num_layers = random.choice(hyperparameter_space['num_layers'])
    learning_rate = random.choice(hyperparameter_space['learning_rate'])
    batch_size = random.choice(hyperparameter_space['batch_size'])

    # Create model with sampled hyperparameters
    model = CNN_LSTM(input_size=39, hidden_size=hidden_size, num_layers=num_layers, output_size=1)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Training loop with current hyperparameters
    num_epochs = 30
    train_losses = []
    test_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        # Shuffle training data
        random_indices = np.random.permutation(len(train_X))
        train_X, train_y, train_seq_lengths = train_X[random_indices], train_y[random_indices], train_seq_lengths[random_indices]

        for i in range(0, len(train_X), batch_size):
            batch_X = train_X[i:i + batch_size]
            batch_y = train_y[i:i + batch_size]
            batch_seq_lengths = train_seq_lengths[i:i + batch_size]

            optimizer.zero_grad()
            output = model(batch_X, batch_seq_lengths)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_X)
        train_losses.append(avg_loss)

        # Validation step
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for i in range(len(val_X)):
                test_X1 = val_X[i].unsqueeze(0)
                test_y1 = val_y[i].unsqueeze(0)
                test_seq_length = val_seq_lengths[i].unsqueeze(0)
                test_output = model(test_X1, test_seq_length)
                test_loss = criterion(test_output, test_y1)
                total_test_loss += test_loss.item()

        avg_test_loss = total_test_loss / len(val_X)
        test_losses.append(avg_test_loss)

    # Check if current hyperparameters are the best
    if avg_test_loss < best_val_loss:
        best_val_loss = avg_test_loss
        best_hyperparams = {
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'learning_rate': learning_rate,
            'batch_size': batch_size
        }

# Print best results
print(f"Best Validation Loss: {best_val_loss}")
print("Best Hyperparameters:", best_hyperparams)


d:\EID-Download\02 Academic\Python38\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\EID-Download\02 Academic\Python38\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
d:\EID-Download\02 Academic\Python38\lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have th

Best Validation Loss: 0.24071228628357252
Best Hyperparameters: {'hidden_size': 32, 'num_layers': 3, 'learning_rate': 0.01, 'batch_size': 8}


Test the model

In [ ]:
# Define other parameters and hyperparameters
input_size = 39  # Feature size
hidden_size = 32
num_layers = 3
batch_size = 8
learning_rate = 0.01
output_size = 1 
num_epochs = 30

# Create the CNN_LSTM model instance
model = CNN_LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, output_size=output_size)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=(0.5, 0.999))
criterion = nn.MSELoss()

#transfer data to torch data
# Convert training data to PyTorch tensors
train_X_new = torch.tensor(train_X_data, dtype=torch.float32)  # Convert train_X to float32 tensor
train_y_new = torch.tensor(train_y_data, dtype=torch.float32)  # Convert train_y to float32 tensor

# Convert testing data to PyTorch tensors
test_X_new = torch.tensor(test_X_data, dtype=torch.float32)    # Convert test_X to float32 tensor
test_y_new = torch.tensor(test_y_data, dtype=torch.float32)    # Convert test_y to float32 tensor

# Generate synthetic data
train_X = train_X_new  # (samples, sequence length, features)
train_y = train_y_new  # Corresponding labels
train_seq_lengths = torch.randint(1, 110, (100,))  # Sequence lengths

test_X = test_X_new
test_y = test_y_new
test_seq_lengths = torch.randint(1, 110, (20,))

train_losses = []
test_losses = []

print("Start Training")

# Training loop
for epoch in range(num_epochs):
    # Shuffle the training data
    random_indices = np.random.permutation(len(train_X))
    train_X = train_X[random_indices]
    train_y = train_y[random_indices]
    train_seq_lengths = train_seq_lengths[random_indices]

    model.train()
    total_loss = 0

    for i in range(len(train_X)):
        # Select a single sample (batch size = 1)
        train_X1 = train_X[i].unsqueeze(0)  # Shape (1, seq_len, features)
        train_y1 = train_y[i].unsqueeze(0)  # Shape (1, output_size)
        seq_length = train_seq_lengths[i].unsqueeze(0)  # Shape (1,)

        # Forward pass
        optimizer.zero_grad()
        output = model(train_X1, seq_length)
        loss = criterion(output, train_y1)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_X)
    train_losses.append(avg_loss)

    # Validation
    model.eval()
    total_test_loss = 0
    with torch.no_grad():
        for i in range(len(test_X)):
            test_X1 = test_X[i].unsqueeze(0)
            test_y1 = test_y[i].unsqueeze(0)
            test_seq_length = test_seq_lengths[i].unsqueeze(0)

            test_output = model(test_X1, test_seq_length)
            test_loss = criterion(test_output, test_y1)
            total_test_loss += test_loss.item()

    avg_test_loss = total_test_loss / len(test_X)
    test_losses.append(avg_test_loss)

    # Log the progress
    if epoch % 1 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {avg_loss:.4f}, Test Loss: {avg_test_loss:.4f}")


Start Training


RuntimeError: Given groups=1, weight of size [39, 39, 2], expected input[1, 41, 109] to have 39 channels, but got 41 channels instead